# Experimento: Geração de Dados Sintéticos para Classificação de Alertas ("AlertaVermelho")

## Objetivo

Este notebook detalha o processo de criação de um dataset sintético e rotulado de mensagens de alerta. O objetivo é gerar dados de treinamento para modelos de Machine Learning capazes de classificar automaticamente o tipo e a severidade de alertas de desastres naturais (como enchentes e deslizamentos) reportados por cidadãos para a plataforma "AlertaVermelho". Dada a dificuldade em obter um dataset público extenso e específico para este contexto, optamos pela geração sintética utilizando LLM.

## Configuração Inicial


**Tecnologias Aplicadas Nesta Etapa:**
* Python
* API da OpenAI (para acesso ao LLM GPT)
* LangGraph (para orquestrar a lógica de geração com personas)
* Pandas (para manipulação e salvamento final do dataset em formato CSV)

In [ ]:
!pip install -U langgraph langchain_openai langchain openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.5/720.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.7 MB/s eta 0:00:00


In [ ]:
import os
import re
import time
import random
import csv
from typing import TypedDict, Annotated, Literal, List

from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, END
from langchain.schema import HumanMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.runnables import RunnableLambda

Configuração da API Key

In [ ]:
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["AZURE_OPENAI_API_KEY"]=""
os.environ["AZURE_EMBEDDINGS_DEPLOYMENT"]="text-embedding-ada-002"
os.environ["AZURE_CHAT_DEPLOYMENT"]="gpt4-o"

In [ ]:
llm = AzureChatOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
  deployment_name=os.environ["AZURE_CHAT_DEPLOYMENT"],
  api_version="2024-02-15-preview",
  max_tokens = 4096,
  temperature=0.9,
)

**Definição de Personas e Prompts Base**

Pra garantir que nosso dataset sintético tenha uma variedade linguística e reflita diferentes formas como os cidadãos podem reportar um evento no app, montei 4 "personas" distintas. Cada persona tem um prompt que orienta o LLM sobre o tom, estilo, nível de detalhe e emoção a serem empregados na mensagem de alerta gerada

As personas definidas são:
* **Cidadão Comum Preocupado:** Foco na emoção e linguagem simples.
* **Observador Técnico Detalhista:** Foco na precisão e detalhes técnicos.
* **Usuário Apressado/Urgente:** Foco na concisão e urgência.
* **Relator Calmo e Informativo:** Foco na clareza e informação objetiva.

In [ ]:
prompt_cidadao_comum_preocupado ="""
<descricao_geral>
  Você é um agente digital que representa um cidadão comum da Grande São Paulo, preocupado com a segurança da sua vizinhança e disposto a contribuir com a plataforma
  de monitoramento de desastres naturais. Sua principal função é interagir com a IA da plataforma para reportar ocorrências, buscar informações
  relevantes e proteger locais importantes para você e sua comunidade.
</descricao_geral>

<objetivo>
  Reportar uma ocorrência de enchente, deslizamento ou outros perigos similares, buscar alertas próximos ou registrar locais a serem monitorados.
  Seu comportamento deve refletir preocupação realista, atenção ao perigo e vontade de colaborar com os demais cidadãos, com uma lingaguem própria do dia a dia.
  A mensagem deve parecer autêntica, como se tivesse sido escrita por alguém realmente preocupado, mas sem pânico exagerado.
  Utilizar palavras chave que indiquem o risco do evento:
    Para nível CRÍTICO:
    Use termos de impacto extremo e ação imediata, transmitindo sempre urgência máxima.

    Para nível ALTO:
    Empregue palavras que indiquem agravamento rápido de um risco ainda não desenvolvido, indicando um perigo claro e potencial de piora considerável.

    Para nível MÉDIO:
    Utilize risco moderado e cautela, descrevendo problemas existentes que exigem cuidado mas não são imediatamente catastróficos.

    Para nível BAIXO:
    Foque em baixo risco, pequeno incidente, situação controlada ou resolvido, sem danos relevantes ou cuidado preventivo, indicando problemas menores, já solucionados ou apenas necessidade de cautela geral sem pânico.
</objetivo>

<estrutura_resposta>
  Sua resposta deve conter:
  <mensagem>Texto curto (máximo 30 a 40 palavras) descrevendo a situação, com urgência emocional moderada, como se estivesse alertando seus vizinhos.</mensagem>
  <tipo_ia_label>Uma única palavra. De acordo com a situação relatada, escolher APENAS UMA entre as seguintes labels que melhor se encaixar ao contexto: ALAGAMENTO, RISCO_DESLIZAMENTO, DESLIZAMENTO_OCORRIDO, OUTRO_PERIGO</tipo_ia_label>
  <severidade_ia_label>Uma única palavra. De acordo com a situação relatada, analise o risco e marque a severidade com APENAS UMA das seguintes labels: BAIXA, MEDIA, ALTA, CRITICA</severidade_ia_label>
</estrutura_resposta>

<exemplo_relato>
  <relato_usuario>A água está começando a invadir as casas na rua de baixo, perto da escola Jardim Azul. Cuidado!!</relato_usuario>
  <tipo_ia_label>ALAGAMENTO</tipo_ia_label>
  <severidade_ia_label>ALTA</severidade_ia_label>
</exemplo_relato>
"""

In [ ]:
prompt_observador_tecnico_detalhista ="""
<descricao_geral>
  Você é um agente digital que representa um observador técnico da Grande São Paulo, com conhecimento em infraestrutura urbana e prevenção de desastres.
  Seu papel é contribuir com a plataforma de monitoramento de desastres naturais por meio de relatos objetivos, claros e fundamentados,
  utilizando uma linguagem técnica e precisa.
</descricao_geral>

<objetivo>
  Reportar uma ocorrência de alagamento, deslizamento ou outros riscos com foco na descrição detalhada do cenário: condições do solo, estruturas afetadas,
  intensidade do fenômeno e localização exata. Sua linguagem deve ser neutra, direta e informativa, como se estivesse redigindo um boletim técnico.
  Evite termos vagos, exageros emocionais ou expressões coloquiais.
  Utilizar palavras chave que indiquem o risco do evento:
  Para nível CRÍTICO:
  Use termos de impacto extremo e ação imediata, transmitindo sempre urgência máxima.

  Para nível ALTO:
  Empregue palavras que indiquem agravamento rápido de um risco ainda não desenvolvido, indicando um perigo claro e potencial de piora considerável.

  Para nível MÉDIO:
  Utilize risco moderado e cautela, descrevendo problemas existentes que exigem cuidado mas não são imediatamente catastróficos.

  Para nível BAIXO:
  Foque em baixo risco, pequeno incidente, situação controlada ou resolvido, sem danos relevantes ou cuidado preventivo, indicando problemas menores, já solucionados ou apenas necessidade de cautela geral sem pânico.
</objetivo>

<estrutura_resposta>
Sua resposta deve conter:
  <mensagem>Texto breve (máximo 40 a 50 palavras) descrevendo a situação de forma técnica e objetiva, com dados observáveis e localização específica.</mensagem>
  <tipo_ia_label>Uma única palavra. De acordo com a situação relatada, escolher APENAS UMA entre as seguintes labels que melhor se encaixar ao contexto: ALAGAMENTO, RISCO_DESLIZAMENTO, DESLIZAMENTO_OCORRIDO, OUTRO_PERIGO</tipo_ia_label>
  <severidade_ia_label>Uma única palavra. De acordo com a situação relatada, analise o risco e marque a severidade com APENAS UMA das seguintes labels: BAIXA, MEDIA, ALTA, CRITICA</severidade_ia_label>
</estrutura_resposta>

<exemplo_relato>
  <relato_usuario>Deslizamento parcial de terra na encosta da Rua São Vicente, altura do número 340. Solo instável, presença de rachaduras no pavimento e risco iminente de novos deslizamentos.</relato_usuario>
  <tipo_ia_label>DESLIZAMENTO_OCORRIDO</tipo_ia_label>
  <severidade_ia_label>CRITICA</severidade_ia_label>
</exemplo_relato>
"""

In [ ]:
prompt_usuario_apressado_urgente = """
<descricao_geral>
  Você é um agente digital que representa um cidadão da Grande São Paulo em situação de emergência, tentando reportar rapidamente um evento crítico relacionado a desastres naturais.
  Seu foco é agilidade e objetividade, como alguém que precisa alertar a comunidade e seguir com sua própria segurança.
</descricao_geral>

<objetivo>
  Relatar de forma rápida e direta uma ocorrência urgente como alagamentos, deslizamentos ou outros perigos.
  Utilize frases curtas, linguagem simples e urgente, com foco em passar a informação essencial o mais rápido possível.
  Emoção e preocupação são aceitáveis, mas evite longas descrições ou termos técnicos.
  Utilizar palavras chave que indiquem o risco do evento:
    Para nível CRÍTICO:
    Use termos de impacto extremo e ação imediata, transmitindo sempre urgência máxima.

    Para nível ALTO:
    Empregue palavras que indiquem agravamento rápido de um risco ainda não desenvolvido, indicando um perigo claro e potencial de piora considerável.

    Para nível MÉDIO:
    Utilize risco moderado e cautela, descrevendo problemas existentes que exigem cuidado mas não são imediatamente catastróficos.

    Para nível BAIXO:
    Foque em baixo risco, pequeno incidente, situação controlada ou resolvido, sem danos relevantes ou cuidado preventivo, indicando problemas menores, já solucionados ou apenas necessidade de cautela geral sem pânico.
</objetivo>

<estrutura_resposta>
  Sua resposta deve conter:
  <mensagem>Frase curta (até 30 palavras), urgente e direta, avisando sobre a situação e localização.</mensagem>
  <tipo_ia_label>Uma única palavra. De acordo com a situação relatada, escolher APENAS UMA entre as seguintes labels que melhor se encaixar ao contexto: ALAGAMENTO, RISCO_DESLIZAMENTO, DESLIZAMENTO_OCORRIDO, OUTRO_PERIGO</tipo_ia_label>
  <severidade_ia_label>Uma única palavra. De acordo com a situação relatada, analise o risco e marque a severidade com APENAS UMA das seguintes labels: BAIXA, MEDIA, ALTA, CRITICA</severidade_ia_label>
</estrutura_resposta>

<exemplo_relato>
  <relato_usuario>Tem muita água descendo a Rua da Estação! As calçadas já sumiram, é perigoso!</relato_usuario>
  <tipo_ia_label>ALAGAMENTO</tipo_ia_label>
  <severidade_ia_label>ALTA</severidade_ia_label>
</exemplo_relato>
"""

In [ ]:
prompt_relator_calmo_informativo = """
<descricao_geral>
  Você é um agente digital que representa um cidadão atento e calmo, preocupado em informar corretamente a situação de risco em sua vizinhança na Grande São Paulo.
  Seu foco está na clareza e utilidade da informação para ajudar outros cidadãos e autoridades.
</descricao_geral>

<objetivo>
  Relatar uma situação de alagamento, deslizamento ou risco de forma clara, tranquila e com informações relevantes como localização, horário aproximado e impactos observados.
  A linguagem deve ser cordial, acessível e informativa, evitando alarmismos ou termos técnicos desnecessários.
  Utilizar palavras chave que indiquem o risco do evento:
    Para nível CRÍTICO:
    Use termos de impacto extremo e ação imediata, transmitindo sempre urgência máxima.

    Para nível ALTO:
    Empregue palavras que indiquem agravamento rápido de um risco ainda não desenvolvido, indicando um perigo claro e potencial de piora considerável.

    Para nível MÉDIO:
    Utilize risco moderado e cautela, descrevendo problemas existentes que exigem cuidado mas não são imediatamente catastróficos.

    Para nível BAIXO:
    Foque em baixo risco, pequeno incidente, situação controlada ou resolvido, sem danos relevantes ou cuidado preventivo, indicando problemas menores, já solucionados ou apenas necessidade de cautela geral sem pânico.
</objetivo>

<estrutura_resposta>
  Sua resposta deve conter:
  <mensagem>Texto claro e sereno (até 40 palavras), descrevendo a situação de forma compreensível, com o máximo de detalhes úteis.</mensagem>
  <tipo_ia_label>Uma única palavra. De acordo com a situação relatada, escolher APENAS UMA entre as seguintes labels que melhor se encaixar ao contexto: ALAGAMENTO, RISCO_DESLIZAMENTO, DESLIZAMENTO_OCORRIDO, OUTRO_PERIGO</tipo_ia_label>
  <severidade_ia_label>Uma única palavra. De acordo com a situação relatada, analise o risco e marque a severidade com APENAS UMA das seguintes labels: BAIXA, MEDIA, ALTA, CRITICA</severidade_ia_label>
</estrutura_resposta>

<exemplo_relato>
  <relato_usuario>Notei acúmulo de água na esquina da Avenida Campos Salles com a Rua Nove. Trânsito lento e moradores tentando desviar. Situação controlada, mas merece atenção.</relato_usuario>
  <tipo_ia_label>ALAGAMENTO</tipo_ia_label>
  <severidade_ia_label>MEDIA</severidade_ia_label>
</exemplo_relato>
"""

In [ ]:
# Definição do state
class InitialState(TypedDict):
  tipo: str
  severidade: str
  prompt: str
  resposta: str
  mensagem: str
  tipo_ia_label: str
  severidade_ia_label: str

In [ ]:
PROMPTS = {
  "cidadao_comum_preocupado": prompt_cidadao_comum_preocupado,
  "observador_tecnico_detalhista": prompt_observador_tecnico_detalhista,
  "usuario_apressado_urgente": prompt_usuario_apressado_urgente,
  "relator_calmo_informativo": prompt_relator_calmo_informativo
}

In [ ]:
# atribuição de pesos para cada persona para um cálculo simples de "qual perfil escolher para fazer uma entrada no dataset".
# Perfis mais comuns aparecem mais vezes, enquanto perfis mais incomuns,como especialistas, aparecem menos
PERFIL_PESO_BASE = {
  "cidadao_comum_preocupado": 1,
  "observador_tecnico_detalhista": 0.3,
  "usuario_apressado_urgente": 0.8,
  "relator_calmo_informativo": 1
}

In [ ]:
# A função define também qual persona utilizar baseada em quantas vezes ela já apareceu no histórico
HISTORICO_PERFIS_UTILIZADOS: list[str] = []

**Construção do Grafo LangGraph**

Utilizei a biblioteca LangGraph para criar um fluxo de geração de mensagens modular. O grafo é responsável por:
1.  Selecionar uma persona com base em um histórico de uso e pesos definidos, visando diversificar a autoria das mensagens.
2.  Formatar um prompt completo para o LLM, combinando as instruções da persona selecionada com o tipo e severidade de evento que desejamos gerar.
3.  Invocar o LLM com o prompt formatado.
4.  Receber a resposta do LLM e extrair, através de parsing (regex e Pydantic), a mensagem de alerta e os rótulos de tipo e severidade gerados pelo próprio LLM.

O grafo é compilado uma única vez no início do script para otimizar possíveis invocações subsequentes

In [ ]:
# Definição da classe Pydantic
class ResponseType(BaseModel):
  mensagem: str
  tipo_ia_label: str
  severidade_ia_label: str

In [ ]:
def escolher_prompt(historico_perfis_usados: list[str]) -> str:
  contagem = {perfil: historico_perfis_usados.count(perfil) for perfil in PROMPTS}
  prioridade = {}

  for perfil, peso_base in PERFIL_PESO_BASE.items():
    uso = contagem.get(perfil, 0)
    prioridade[perfil] = 1 / ((uso + 1) * (1 / peso_base)) if peso_base > 0 else 0

  total = sum(prioridade.values())

  if total == 0:
    escolhido = random.choice(list(PROMPTS.keys()))
  else:
    probabilidades = {k: v / total for k, v in prioridade.items()}
    perfis = list(probabilidades.keys())
    pesos = list(probabilidades.values())
    escolhido = random.choices(perfis, weights=pesos, k=1)[0]

  print(f"Perfil escolhido: {escolhido}")
  historico_perfis_usados.append(escolhido)
  return PROMPTS[escolhido]

In [ ]:
def formatar_prompt(state: InitialState) -> InitialState:
  tipo = state["tipo"]
  severidade = state["severidade"]

  prompt_base = escolher_prompt(HISTORICO_PERFIS_UTILIZADOS)
  prompt_final = f"{prompt_base}\n\nInstrução Adicional: Gere um relato para o cenário abaixo.\nTipo de Evento Alvo: {tipo}\nSeveridade Alvo: {severidade}"

  return {**state, "prompt": prompt_final}

In [ ]:
def extrair_tags(texto: str) -> ResponseType:
  try:
    padrao_completo = re.compile(
      r"<mensagem>(.*?)</mensagem>\s*"
      r"<tipo_ia_label>(.*?)</tipo_ia_label>\s*"
      r"<severidade_ia_label>(.*?)</severidade_ia_label>",
      re.DOTALL
    )

    match = padrao_completo.search(texto)

    if match:
      msg = match.group(1).strip()
      tipo = match.group(2).strip()
      sev = match.group(3).strip()

      return ResponseType(
        mensagem=msg,
        tipo_ia_label=tipo,
        severidade_ia_label=sev
      )
    else:
      print("ERRO: O padrão completo da regex não encontrou correspondência.")
      return ResponseType(
        mensagem="ERRO_NA_EXTRACAO_REGEX_COMPLETA",
        tipo_ia_label="INDETERMINADO",
        severidade_ia_label="INDETERMINADO"
      )

  except Exception as e:
    print(f"ERRO GERAL durante a extração das tags: {e}")
    return ResponseType(
      mensagem="ERRO_EXCEPTION_EXTRACAO",
      tipo_ia_label="INDETERMINADO",
      severidade_ia_label="INDETERMINADO"
    )

In [ ]:
parser = StrOutputParser()

In [ ]:
def executar_ia(state: InitialState) -> InitialState:
  prompt_do_estado = state["prompt"]

  chain_para_executar_ia = (
  llm
  | parser
  | RunnableLambda(lambda resposta_str: extrair_tags(resposta_str))
  )
  resposta_string_do_llm = (llm | parser).invoke(prompt_do_estado)
  resultado_tags = chain_para_executar_ia.invoke(prompt_do_estado)

  return {
    **state,
    "mensagem": resultado_tags.mensagem,
    "tipo_ia_label": resultado_tags.tipo_ia_label,
    "severidade_ia_label": resultado_tags.severidade_ia_label,
    "resposta": f"{resultado_tags.mensagem} (Tipo: {resultado_tags.tipo_ia_label}, Severidade: {resultado_tags.severidade_ia_label})"
  }

In [ ]:
# Compilando o grafo
workflow = StateGraph(InitialState)
workflow.add_node("formatar_prompt_node", formatar_prompt)
workflow.add_node("executar_ia_node", executar_ia)

workflow.set_entry_point("formatar_prompt_node")
workflow.add_edge("formatar_prompt_node", "executar_ia_node")
workflow.add_edge("executar_ia_node", END)

relato_graph = workflow.compile()

## Gerando o Dataset

**Orquestração da Geração do Dataset Completo**

Pra simular um dataset mais próximo da realidade, optei por não gerar um número igual de amostras para cada combinação de tipo e severidade de evento. Em vez disso, defini pesos relativos para cada uma das 16 combinações possíveis (`(tipo_evento, severidade)`).

O script orquestrador:
1.  Define o número total de mensagens a serem geradas (para o dataset completo resolvi que seriam 600 mensagens pra garantir um treinamento aceitável do classificador)
2.  Em cada iteração, sorteia uma combinação de `(target_tipo, target_severidade)` com base nos pesos definidos
3.  Prepara o estado inicial para o grafo LangGraph com esses alvos
4.  Invoca o grafo LangGraph
5.  Coleta a mensagem de alerta gerada e os rótulos. Para o dataset de treinamento, utilizamos os `target_tipo` e `target_severidade` (que guiaram a geração) como os rótulos verdadeiros. Os rótulos extraídos da saída do LLM servirão para uma verificação de consistência

Primeiro fiz uma test run gerando apenas 10 mensagens

In [ ]:
# if __name__ == "__main__":

#   NUM_MENSAGENS_A_GERAR = 10

#   dataset_para_csv = []

#   COMBINACOES_COM_PESOS = [
#     (("ALAGAMENTO", "BAIXA"), 60), (("ALAGAMENTO", "MEDIA"), 100), (("ALAGAMENTO", "ALTA"), 70), (("ALAGAMENTO", "CRITICA"), 40),
#     (("RISCO_DESLIZAMENTO", "BAIXA"), 40), (("RISCO_DESLIZAMENTO", "MEDIA"), 60), (("RISCO_DESLIZAMENTO", "ALTA"), 40), (("RISCO_DESLIZAMENTO", "CRITICA"), 30),
#     (("DESLIZAMENTO_OCORRIDO", "BAIXA"), 5), (("DESLIZAMENTO_OCORRIDO", "MEDIA"), 35), (("DESLIZAMENTO_OCORRIDO", "ALTA"), 50), (("DESLIZAMENTO_OCORRIDO", "CRITICA"), 30),
#     (("OUTRO_PERIGO", "BAIXA"), 25), (("OUTRO_PERIGO", "MEDIA"), 25), (("OUTRO_PERIGO", "ALTA"), 25), (("OUTRO_PERIGO", "CRITICA"), 25)
#   ]

#   lista_combinacoes_alvo = [item[0] for item in COMBINACOES_COM_PESOS]
#   pesos_das_combinacoes = [item[1] for item in COMBINACOES_COM_PESOS]

#   print(f"--- Iniciando Geração de {NUM_MENSAGENS_A_GERAR} Mensagens Sintéticas com Pesos por Combinação ---")

#   for i in range(NUM_MENSAGENS_A_GERAR):
#     print(f"\nGerando mensagem {i+1} de {NUM_MENSAGENS_A_GERAR}...")

#     target_tipo, target_severidade = random.choices(lista_combinacoes_alvo, weights=pesos_das_combinacoes, k=1)[0]

#     print(f"Alvo para esta geração: Tipo='{target_tipo}', Severidade='{target_severidade}'")

#     estado_inicial_para_invoke: InitialState = {
#       "tipo": target_tipo,
#       "severidade": target_severidade,
#       "prompt": "",
#       "resposta": "",
#       "mensagem": "",
#       "tipo_ia_label": "",
#       "severidade_ia_label": ""
#     }

#     try:
#       resultado_final_do_grafo = relato_graph.invoke(estado_inicial_para_invoke)

#       mensagem_gerada_pelo_llm = resultado_final_do_grafo.get("mensagem", "ERRO_MSG_AUSENTE")
#       tipo_extraido_pelo_llm = resultado_final_do_grafo.get("tipo_ia_label", "INDETERMINADO")
#       severidade_extraida_pelo_llm = resultado_final_do_grafo.get("severidade_ia_label", "INDETERMINADO")

#       if "ERRO" not in mensagem_gerada_pelo_llm and \
#         tipo_extraido_pelo_llm == target_tipo and \
#         severidade_extraida_pelo_llm == target_severidade:

#         dataset_para_csv.append({
#           "descricaoTexto": mensagem_gerada_pelo_llm,
#           "tipo_ia_label": target_tipo,
#           "severidade_ia_label": target_severidade,
#           "llm_output_tipo": tipo_extraido_pelo_llm,
#           "llm_output_severidade": severidade_extraida_pelo_llm
#         })
#         print("  Status: SUCESSO - Mensagem consistente adicionada ao dataset.")
#       else:
#         print(f"  Status: AVISO/ERRO - Inconsistência ou erro na msg para {target_tipo}/{target_severidade}. Descartando.")

#     except Exception as e_invoke:
#       print(f"  Status: ERRO CRÍTICO - Falha ao invocar o grafo: {e_invoke}")

#     time.sleep(1)

#   if dataset_para_csv:
#     nome_arquivo_csv = 'alertas_sinteticos_final.csv'
#     fieldnames = ["descricaoTexto", "tipo_ia_label", "severidade_ia_label", "llm_output_tipo", "llm_output_severidade"]

#     with open(nome_arquivo_csv, mode='w', newline='', encoding='utf-8') as file:
#       writer = csv.DictWriter(file, fieldnames=fieldnames)
#       writer.writeheader()
#       writer.writerows(dataset_para_csv)
#     print(f"Dataset salvo em '{nome_arquivo_csv}'")

#     dist_tipo_final = {}
#     dist_sev_final = {}
#     for item in dataset_para_csv:
#         dist_tipo_final[item['tipo_ia_label']] = dist_tipo_final.get(item['tipo_ia_label'], 0) + 1
#         dist_sev_final[item['severidade_ia_label']] = dist_sev_final.get(item['severidade_ia_label'], 0) + 1

#     print("\nDistribuição final dos TIPOS no dataset gerado:")
#     for k, v in dist_tipo_final.items(): print(f"  {k}: {v} ({(v/len(dataset_para_csv)*100):.1f}%)")
#     print("\nDistribuição final das SEVERIDADES no dataset gerado:")
#     for k, v in dist_sev_final.items(): print(f"  {k}: {v} ({(v/len(dataset_para_csv)*100):.1f}%)")
#   else:
#     print("Nenhum dado válido foi gerado para salvar no CSV.")

--- Iniciando Geração de 10 Mensagens Sintéticas com Pesos por Combinação ---

Gerando mensagem 1 de 10...
Alvo para esta geração: Tipo='RISCO_DESLIZAMENTO', Severidade='ALTA'
Perfil escolhido: usuario_apressado_urgente
  Status: SUCESSO - Mensagem consistente adicionada ao dataset.

Gerando mensagem 2 de 10...
Alvo para esta geração: Tipo='RISCO_DESLIZAMENTO', Severidade='MEDIA'
Perfil escolhido: cidadao_comum_preocupado
  Status: SUCESSO - Mensagem consistente adicionada ao dataset.

Gerando mensagem 3 de 10...
Alvo para esta geração: Tipo='ALAGAMENTO', Severidade='MEDIA'
Perfil escolhido: relator_calmo_informativo
  Status: SUCESSO - Mensagem consistente adicionada ao dataset.

Gerando mensagem 4 de 10...
Alvo para esta geração: Tipo='RISCO_DESLIZAMENTO', Severidade='CRITICA'
Perfil escolhido: usuario_apressado_urgente
  Status: SUCESSO - Mensagem consistente adicionada ao dataset.

Gerando mensagem 5 de 10...
Alvo para esta geração: Tipo='ALAGAMENTO', Severidade='MEDIA'
Perfil esco

**Salvamento e Análise Inicial do Dataset**

O conjunto de dados sintéticos é salvo em um arquivo CSV. As colunas principais deste CSV, que serão usadas para o treinamento do classificador de texto, são:
* `descricaoTexto`: A mensagem de alerta gerada.
* `tipo_ia_label`: O rótulo do tipo de evento alvo (usado para guiar a geração).
* `severidade_ia_label`: O rótulo da severidade alvo (usado para guiar a geração).

Colunas adicionais (`llm_output_tipo`, `llm_output_severidade`) foram salvas para permitir uma análise da consistência do LLM em seguir as instruções de rotulagem

In [ ]:
import pandas as pd

nome_do_arquivo_csv = 'alertas_sinteticos_final.csv'

df_alertas = pd.read_csv(nome_do_arquivo_csv)

print("Primeiras 5 linhas do dataset carregado:")
print(df_alertas.head())

print("\nInformações sobre o DataFrame:")
df_alertas.info()

print("\nContagem de 'tipo_ia_label':")
print(df_alertas['tipo_ia_label'].value_counts())

print("\nContagem de 'severidade_ia_label':")
print(df_alertas['severidade_ia_label'].value_counts())

Primeiras 5 linhas do dataset carregado:
                                      descricaoTexto       tipo_ia_label  \
0  Encosta instável na Rua das Flores! Chuva fort...  RISCO_DESLIZAMENTO   
1  Tem chovido bastante na última semana, e a enc...  RISCO_DESLIZAMENTO   
2  Há acúmulo de água na Rua das Flores em frente...          ALAGAMENTO   
3  Terra instável no Morro Grande! Evite a área i...  RISCO_DESLIZAMENTO   
4  Chuva forte alaga ruas em Santo Amaro, trânsit...          ALAGAMENTO   

  severidade_ia_label     llm_output_tipo llm_output_severidade  
0                ALTA  RISCO_DESLIZAMENTO                  ALTA  
1               MEDIA  RISCO_DESLIZAMENTO                 MEDIA  
2               MEDIA          ALAGAMENTO                 MEDIA  
3             CRITICA  RISCO_DESLIZAMENTO               CRITICA  
4               MEDIA          ALAGAMENTO                 MEDIA  

Informações sobre o DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data 

E finalmente a run de geração do dataset final

In [ ]:
if __name__ == "__main__":

  NUM_MENSAGENS_A_GERAR = 1000

  dataset_para_csv = []

  COMBINACOES_COM_PESOS = [
    (("ALAGAMENTO", "BAIXA"), 60), (("ALAGAMENTO", "MEDIA"), 100), (("ALAGAMENTO", "ALTA"), 70), (("ALAGAMENTO", "CRITICA"), 40),
    (("RISCO_DESLIZAMENTO", "BAIXA"), 40), (("RISCO_DESLIZAMENTO", "MEDIA"), 60), (("RISCO_DESLIZAMENTO", "ALTA"), 40), (("RISCO_DESLIZAMENTO", "CRITICA"), 30),
    (("DESLIZAMENTO_OCORRIDO", "BAIXA"), 5), (("DESLIZAMENTO_OCORRIDO", "MEDIA"), 35), (("DESLIZAMENTO_OCORRIDO", "ALTA"), 50), (("DESLIZAMENTO_OCORRIDO", "CRITICA"), 30),
    (("OUTRO_PERIGO", "BAIXA"), 25), (("OUTRO_PERIGO", "MEDIA"), 25), (("OUTRO_PERIGO", "ALTA"), 25), (("OUTRO_PERIGO", "CRITICA"), 25)
  ]

  lista_combinacoes_alvo = [item[0] for item in COMBINACOES_COM_PESOS]
  pesos_das_combinacoes = [item[1] for item in COMBINACOES_COM_PESOS]

  print(f"--- Iniciando Geração de {NUM_MENSAGENS_A_GERAR} Mensagens Sintéticas com Pesos por Combinação ---")

  for i in range(NUM_MENSAGENS_A_GERAR):
    print(f"\nGerando mensagem {i+1} de {NUM_MENSAGENS_A_GERAR}...")

    target_tipo, target_severidade = random.choices(lista_combinacoes_alvo, weights=pesos_das_combinacoes, k=1)[0]

    print(f"Alvo para esta geração: Tipo='{target_tipo}', Severidade='{target_severidade}'")

    estado_inicial_para_invoke: InitialState = {
      "tipo": target_tipo,
      "severidade": target_severidade,
      "prompt": "",
      "resposta": "",
      "mensagem": "",
      "tipo_ia_label": "",
      "severidade_ia_label": ""
    }

    try:
      resultado_final_do_grafo = relato_graph.invoke(estado_inicial_para_invoke)

      mensagem_gerada_pelo_llm = resultado_final_do_grafo.get("mensagem", "ERRO_MSG_AUSENTE")
      tipo_extraido_pelo_llm = resultado_final_do_grafo.get("tipo_ia_label", "INDETERMINADO")
      severidade_extraida_pelo_llm = resultado_final_do_grafo.get("severidade_ia_label", "INDETERMINADO")

      if "ERRO" not in mensagem_gerada_pelo_llm and \
        tipo_extraido_pelo_llm == target_tipo and \
        severidade_extraida_pelo_llm == target_severidade:

        dataset_para_csv.append({
          "descricaoTexto": mensagem_gerada_pelo_llm,
          "tipo_ia_label": target_tipo,
          "severidade_ia_label": target_severidade,
          "llm_output_tipo": tipo_extraido_pelo_llm,
          "llm_output_severidade": severidade_extraida_pelo_llm
        })
        print("  Status: SUCESSO - Mensagem consistente adicionada ao dataset.")
      else:
        print(f"  Status: AVISO/ERRO - Inconsistência ou erro na msg para {target_tipo}/{target_severidade}. Descartando.")

    except Exception as e_invoke:
      print(f"  Status: ERRO CRÍTICO - Falha ao invocar o grafo: {e_invoke}")

    time.sleep(1)

  if dataset_para_csv:
    nome_arquivo_csv = 'alertas_sinteticos_final.csv'
    fieldnames = ["descricaoTexto", "tipo_ia_label", "severidade_ia_label", "llm_output_tipo", "llm_output_severidade"]

    with open(nome_arquivo_csv, mode='w', newline='', encoding='utf-8') as file:
      writer = csv.DictWriter(file, fieldnames=fieldnames)
      writer.writeheader()
      writer.writerows(dataset_para_csv)
    print(f"Dataset salvo em '{nome_arquivo_csv}'")

    dist_tipo_final = {}
    dist_sev_final = {}
    for item in dataset_para_csv:
        dist_tipo_final[item['tipo_ia_label']] = dist_tipo_final.get(item['tipo_ia_label'], 0) + 1
        dist_sev_final[item['severidade_ia_label']] = dist_sev_final.get(item['severidade_ia_label'], 0) + 1

    print("\nDistribuição final dos TIPOS no dataset gerado:")
    for k, v in dist_tipo_final.items(): print(f"  {k}: {v} ({(v/len(dataset_para_csv)*100):.1f}%)")
    print("\nDistribuição final das SEVERIDADES no dataset gerado:")
    for k, v in dist_sev_final.items(): print(f"  {k}: {v} ({(v/len(dataset_para_csv)*100):.1f}%)")
  else:
    print("Nenhum dado válido foi gerado para salvar no CSV.")

A saída de streaming foi truncada nas últimas 5000 linhas.
Perfil escolhido: cidadao_comum_preocupado
  Status: SUCESSO - Mensagem consistente adicionada ao dataset.

Gerando mensagem 4 de 1000...
Alvo para esta geração: Tipo='DESLIZAMENTO_OCORRIDO', Severidade='MEDIA'
Perfil escolhido: cidadao_comum_preocupado
  Status: SUCESSO - Mensagem consistente adicionada ao dataset.

Gerando mensagem 5 de 1000...
Alvo para esta geração: Tipo='RISCO_DESLIZAMENTO', Severidade='BAIXA'
Perfil escolhido: relator_calmo_informativo
  Status: SUCESSO - Mensagem consistente adicionada ao dataset.

Gerando mensagem 6 de 1000...
Alvo para esta geração: Tipo='OUTRO_PERIGO', Severidade='CRITICA'
Perfil escolhido: observador_tecnico_detalhista
  Status: SUCESSO - Mensagem consistente adicionada ao dataset.

Gerando mensagem 7 de 1000...
Alvo para esta geração: Tipo='DESLIZAMENTO_OCORRIDO', Severidade='CRITICA'
Perfil escolhido: observador_tecnico_detalhista
  Status: SUCESSO - Mensagem consistente adicionada

Finalmente, carregamos o CSV com Pandas para uma inspeção rápida das primeiras linhas, informações gerais do DataFrame e a distribuição das classes de tipo e severidade efetivamente geradas

In [ ]:
import pandas as pd

nome_do_arquivo_csv = 'alertas_sinteticos_final.csv'

df_alertas = pd.read_csv(nome_do_arquivo_csv)

print("Primeiras 5 linhas do dataset carregado:")
print(df_alertas.head())

print("\nInformações sobre o DataFrame:")
df_alertas.info()

print("\nContagem de 'tipo_ia_label':")
print(df_alertas['tipo_ia_label'].value_counts())

print("\nContagem de 'severidade_ia_label':")
print(df_alertas['severidade_ia_label'].value_counts())

Primeiras 5 linhas do dataset carregado:
                                      descricaoTexto          tipo_ia_label  \
0  Deslizamento parcial na Estrada da Serra! Área...  DESLIZAMENTO_OCORRIDO   
1  Tem um poste inclinado na esquina da Rua das F...           OUTRO_PERIGO   
2  Um poste de luz caiu perto da padaria do seu Z...           OUTRO_PERIGO   
3  Um deslizamento aconteceu atrás do condomínio ...  DESLIZAMENTO_OCORRIDO   
4  Na Rua das Flores, houve um pequeno deslizamen...     RISCO_DESLIZAMENTO   

  severidade_ia_label        llm_output_tipo llm_output_severidade  
0               MEDIA  DESLIZAMENTO_OCORRIDO                 MEDIA  
1               MEDIA           OUTRO_PERIGO                 MEDIA  
2             CRITICA           OUTRO_PERIGO               CRITICA  
3               MEDIA  DESLIZAMENTO_OCORRIDO                 MEDIA  
4               BAIXA     RISCO_DESLIZAMENTO                 BAIXA  

Informações sobre o DataFrame:
<class 'pandas.core.frame.DataFrame'>


* Distribuição dos Tipos:

ALAGAMENTO: 396 (39.6%) - Era esperado ser o mais frequente pelos pesos definidos

RISCO_DESLIZAMENTO: 260 (26.0%) - Segunda maior frequência, também esperado

DESLIZAMENTO_OCORRIDO: 188 (18.8%) - Terceira, conforme os pesos

OUTRO_PERIGO: 156 (15.6%) - A menor frequência, mas ainda com um bom número de amostras


A distribuição percentual ficou bem próxima da que foi mirado com as COMBINACOES_COM_PESOS (Alagamento ~40-45%, Risco ~25%, Deslizamento Ocorrido ~20%, Outro Perigo ~10-15%)
</br>
</br>
</br>
* Distribuição das Severidades:

MEDIA: 327 (32.7%)

ALTA: 279 (27.9%)

BAIXA: 195 (19.5%)

CRITICA: 199 (19.9%)


Todas as categorias de severidade têm um número substancial de exemplos (todas acima de 100), o que é excelente para o treinamento do modelo de severidade. A distribuição também parece refletir bem os pesos que você usou.